In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 20
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset): # ExeDataset不是我取的，要記得改
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)
        
    def forward(self,x):
        embedd_x = self.embedding(x)
        # embedd_x.requires_grad = True
        embedd_x.retain_grad()
        x = embedd_x.transpose(-1,-2)
        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()
        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x,embedd_x

In [10]:
def mp_func(idx,one_of_all_embedd,attacked_embedd_x):
    temp = np.linalg.norm(attacked_embedd_x - one_of_all_embedd,ord=2,axis=-1)
    temp = np.expand_dims(temp, axis=-1)
    return [idx,temp]

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [13]:
for round_ in range(200):
    model.eval()
    total_acc = []
    # bar = tqdm(validloader)
    bar = tqdm(testloader)
    for step, (batch_data,batch_label) in enumerate(bar):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze()

        mask = (batch_data == 0).float()
        random_padding = torch.randint_like(input=mask.float(),low=1,high=257) * mask
        batch_data += random_padding.long()

        optim.zero_grad()
        temp_label = torch.zeros((len(batch_label),2)).cuda()
#         for idx,target in enumerate(batch_label.squeeze()): temp_label[idx,target] = 1
#         temp_label = temp_label.cuda() if CUDA else temp_label
        temp_label[:,0] = 1
        pred,embedd_x = model(batch_data)
        loss = ce_loss(pred, temp_label)
        loss.backward()

        attacked_embedd_x = embedd_x - 0.5*embedd_x.grad.sign()
        all_embedd = model.embedding(torch.arange(start=0,end=257).cuda())

        shape = attacked_embedd_x.shape
        attacked_embedd_x = attacked_embedd_x.reshape((shape[0],shape[1],1,shape[2]))
        attacked_embedd_x = attacked_embedd_x.expand(shape[0],shape[1],257,shape[2])

        results = torch.linalg.norm(attacked_embedd_x - all_embedd,ord=2,dim=-1)
        new_batch_data = ((1-mask)*batch_data + mask*torch.argmin(results,dim=-1)).long()

        pred,embedd_x = model(new_batch_data)
        _, predicted = torch.max(pred, 1)
        acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
        total_acc.append(acc)
        bar.set_description(f'{np.mean(total_acc)}')

#     result_path = "/workdir/security/home/junjiehuang2468/paper/results/ember/append_FSM/"
#     result_name = '_'.join(best_trained_model.split('/'))[:-3] + f'_epoch_{round_}.txt'
#     with open(result_path + result_name,'wb') as fp:
#         pickle.dump(total_acc,fp)


0.0052453027139874745:  38%|███▊      | 1916/5000 [01:10<01:53, 27.10it/s]


KeyboardInterrupt: 